In [ ]:
import os
from pathlib import Path

import syft_rds as sy

## DO loads dataset

In [ ]:
!just reset-sim
!just setup-embedded-devices

In [ ]:
DO = "yash@openmined.org"
DATASET_NAME = "fashionmnist_part_1"

!just dump-sim-config {DO}

In [ ]:
PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / "embedded-devices"
SIM_DIR = PROJECT_DIR / ".sim"
CONFIG_DIR = SIM_DIR / ".config"

print(f"{PROJECT_DIR = }")
print(f"{FLWR_PROJECT_PATH = }")
print(f"{SIM_DIR = }")
print(f"{CONFIG_DIR = }")

In [ ]:
DO_CONFIG = CONFIG_DIR / f"{DO}.json"
!cat $DO_CONFIG

In [ ]:
do = sy.init_session(host=DO, syftbox_client_config_path=DO_CONFIG)
assert do.host == DO
assert do.email == DO
assert do.is_admin

In [ ]:
DATASET_PRIVATE_PATH = FLWR_PROJECT_PATH / "datasets" / DATASET_NAME
DATASET_MOCK_PATH = FLWR_PROJECT_PATH / "datasets" / DATASET_NAME
README_PATH = FLWR_PROJECT_PATH / "README.md"

SYFTBOX_DATASET_NAME = "fashionmnist"
print(f"DO {DO} uploading dataset {SYFTBOX_DATASET_NAME}")

try:
    dataset = do.dataset.create(
        name=SYFTBOX_DATASET_NAME,
        summary="Partitioned FashionMNIST dataset.",
        description_path=README_PATH,
        path=DATASET_PRIVATE_PATH,
        mock_path=DATASET_MOCK_PATH,
    )
    dataset.describe()
except Exception as e:
    print(e)

In [ ]:
print(do.datasets)

In [ ]:
flwr_project_path = str(FLWR_PROJECT_PATH)
mock_path = str(do.dataset.get(name=SYFTBOX_DATASET_NAME).get_mock_path())

## Run syft_flwr simulation
This can be run after the DS bootstrap the `flwr` project (has the `main.py`)

In [ ]:
if (FLWR_PROJECT_PATH / "main.py").exists():
    !just run-sim-client {flwr_project_path} {DO} {mock_path}